In [1]:
from utils.openai_helpers import query_openai_model, query_openai_model_batch
from utils.prompt_functions import generate_verification_prompt
from utils.preprocess_functions import get_labels_and_descriptions_for_triplets
from utils.wiki_helpers import get_info_for_qid
import numpy as np
import pandas as pd
import json
import concurrent.futures
from tqdm import tqdm
import time



In [2]:
from utils.openai_helpers import query_openai_model, query_openai_model_batch

In [3]:
df = pd.read_csv('inputs/KG_Benchmark_may28_results - results.csv')

In [4]:
df.head()

,QUESTION,ITEM_1,ITEM_2,ITEM_3,ITEM_4,PROP_1,PROP_2,PROP_3,ITEM_1_ALIAS,ITEM_2_ALIAS,ITEM_3_ALIAS,ITEM_4_ALIAS,PROP_1_ALIAS,PROP_2_ALIAS,PROP_3_ALIAS
0,What is the profession of the coach of the tea...,Q2723827,Q185173,Q734768,Q5137571,P54,P286,P106,"Lukša Andrić, lukša andrić, Luksa Andric, luks...","cedevita zagreb, kk cedevita zagreb, Cedevita ...","jasmin repesa, Jasmin Repesa, Repesa, Repeša, ...","Head basketball coach, coach (basketball), bas...","member of sports team, member of team, team, t...","head coach, manager, club manager, senior coac...","occupation, profession, job, work, career, emp..."
1,What are the elements that the field of profes...,Q3090093,Q82955,Q7163,Q1711347,P106,P425,P527,"frédéric ngenzebuhoro, frederic ngenzebuhoro, ...","Politician, political figure, career politics,...","political jargon, Political action movement, p...","cultural policies as catalysts of creativity, ...","occupation, profession, job, work, career, emp...","field of this occupation, profession's field, ...","has part, formed from, formed out of, assemble..."
2,What is the history of the alphabet associated...,Q5046202,Q16275174,Q8229,Q3772237,P735,P282,P2184,"Carrie Austin, carrie austin","Carrie (given name), carrie (given name), carr...","Latin alphabet letters, latin alphabet/sandbox...","History of the latin alphabet, history of the ...","given name, forename, first name, personal nam...","writing system, alphabet, script","history of topic, history, timeline of topic, ..."
3,What is the administrative territorial entity ...,Q16018708,Q3044572,Q4979549,Q36687,P54,P115,P131,"jeff clifton, Jeff Clifton","fitzroy fc, fitzroy f.c., Fitzroy Football Clu...","brunswick st oval, Brunswick Street Oval, brun...","VIC, Victoria Australia, vic (australia), AU-V...","member of sports team, member of team, team, t...","home venue, ground, home field, arena, home gr...",located in the administrative territorial enti...
4,What is the administrative territorial entity ...,Q1352535,Q680449,Q28009454,Q700427,P47,P1366,P131,"Bessac, bessac","Blanzac-Porcheresse, blanzac-porcheresse","Coteaux du Blanzacais, coteaux du blanzacais","Arrondissement of Angoulême, arrondissement of...","shares border with, borders, bordered by, adja...","replaced by, heir, next job holder, successor,...",located in the administrative territorial enti...


In [5]:
df.shape

(9383, 15)

In [6]:
df.iloc[3]['ITEM_4_ALIAS']

"VIC, Victoria Australia, vic (australia), AU-VIC, victoria australia, victoria state, australia victoria, victoria (australia), vic, melbourne public libraries, victoria (australian state), Victoria, Australia, Victoria aus, victoria, australia, visit victoria, state of victoria, vic.au, Australia's state of Victoria, VIC (Australia), victoria (state), australia's state of victoria, au-vic, .vic.au, State of Victoria, Victoria (State), Vic.au, Victoria State, Visit Victoria, Victoria (Australian state), victoria aus, Australia Victoria, Victoria (Australia), Victoria (state), Melbourne Public Libraries"

In [7]:
def get_entries(df, idx):
    question = df.iloc[idx]['QUESTION']
    answer = df.iloc[idx]['ITEM_4']
    triplets = [
        (df.iloc[idx]['ITEM_1'], df.iloc[idx]['PROP_1'], df.iloc[idx]['ITEM_2']),
        (df.iloc[idx]['ITEM_2'], df.iloc[idx]['PROP_2'], df.iloc[idx]['ITEM_3']),
        (df.iloc[idx]['ITEM_3'], df.iloc[idx]['PROP_3'], df.iloc[idx]['ITEM_4'])
    ]
    return question, answer, triplets

In [8]:

prompts_dict = {}

def process_index(i, df, retries = 0, max_retries = 5):
    
    while retries < max_retries:
        try:
            req = get_entries(df, i)
            triplet_list, desc_list = get_labels_and_descriptions_for_triplets(req[2])
            label, desc, aliases = get_info_for_qid(req[1])
            
            question = req[0]
            path = ' -> '.join(map(str, triplet_list))
            ans = label
            
            prompt = generate_verification_prompt(question, ans)
            prompts_dict[i] = prompt
            return
        except Exception as e:
            retries += 1
            time.sleep(2 ** retries)  # Exponential backoff
            if retries == max_retries:
                prompts_dict[i] = "Failed"

# Using ThreadPoolExecutor for multi-threading
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Creating a list of futures
    futures = [executor.submit(process_index, i, df) for i in range(500)]

    # Using tqdm to track progress
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        pass


  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/500 [00:00<?, ?it/s]

 39%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 197/500 [01:29<02:30,  2.01it/s]

Error: HTTP Error 404: Not Found
Error: cannot unpack non-iterable NoneType object
Failed to get labels and descriptions for triplet: ('Q16729691', 'P69', 'Q49116')


 41%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 203/500 [01:32<02:39,  1.86it/s]

Error: HTTP Error 404: Not Found
Error: cannot unpack non-iterable NoneType object
Failed to get labels and descriptions for triplet: ('Q2504463', 'P17', 'Q30')


...

 62%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                                                                                                                                                                                                                                                                                                                                                | 308/500 [02:17<01:11,  2.70it/s]

Error: HTTP Error 404: Not Found
Error: cannot unpack non-iterable NoneType object
Failed to get labels and descriptions for triplet: ('Q14624392', 'P1441', 'Q7115636')


 69%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                                                                                                                                                                                                                                                                                         | 345/500 [02:34<01:04,  2.40it/s]

Error: HTTP Error 404: Not Found
Error: cannot unpack non-iterable NoneType object
Failed to get labels and descriptions for triplet: ('Q9176', 'P134', 'Q625800')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [03:41<00:00,  2.26it/s]


In [12]:
# Collecting the keys of all failed attempts
failed_keys = [key for key, value in prompts_dict.items() if value == "Failed"]

# Output the keys of all failed attempts
print("Failed keys:", failed_keys)

Failed keys: []


In [13]:
len(prompts_dict)

500

In [14]:
np.save('outputs/prompts_dict500.npy', prompts_dict)

In [8]:
prompts_dict = {}
for i in range(500):
    req = get_entries(df, i)
    triplet_list, desc_list = get_labels_and_descriptions_for_triplets(req[2])
    label, desc, aliases = get_info_for_qid(req[1])
    
    question = req[0]
    path = ' -> '.join(map(str, triplet_list))
    ans = label
    
    prompt = generate_verification_prompt(question, ans)
    prompts_dict[i] = prompt



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.22it/s]


KeyboardInterrupt: 

In [ ]:
# req = get_entries(df, 3)
# triplet_list, desc_list = get_labels_and_descriptions_for_triplets(req[2])
# label, desc, aliases = get_info_for_qid(req[1])

# question = req[0]
# path = ' -> '.join(map(str, triplet_list))
# ans = label

# # Example usage
# question = "What is the administrative territorial entity where the home venue of the sports team that Jeff Clifton was a member of is located?"
# answer = "Melbourne"
# prompt = generate_verification_prompt(question, answer)
# print(prompt)
# res = query_openai_model(prompt)

In [8]:
req = get_entries(df, 3)

In [9]:
# req = get_entries(df, 772)

In [11]:
triplet_list, desc_list = get_labels_and_descriptions_for_triplets(req[2])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.40it/s]


In [12]:
triplet_list

[('Jeff Clifton', 'member of sports team', 'Fitzroy Football Club'),
 ('Fitzroy Football Club', 'home venue', 'Brunswick Street Oval'),
 ('Brunswick Street Oval',
  'located in the administrative territorial entity',
  'Victoria')]

In [13]:
req[0], req[1]

('What is the administrative territorial entity where the home venue of the sports team that Jeff Clifton was a member of is located?',
 'Q36687')

In [14]:
label, desc, aliases = get_info_for_qid(req[1])

In [15]:
label, desc, aliases

('Victoria',
 'state of Australia',
 ['VIC',
  'Victoria, Australia',
  'State of Victoria',
  'State of VIC',
  'Vic.',
  'AU-VIC'])

In [16]:
question = req[0]
path = ' -> '.join(map(str, triplet_list))
ans = label

In [17]:
# def generate_verification_prompt(question, answer):
#     prompt = f"""
# **Prompt Adequacy Verification Test**

# **Objective:** Ensure the question is coherent, and the provided answer is a suitable and relevant response that addresses the informational needs of the question.

# **Instructions:**

# 1. **Read the Question:**
#    - Does the question make logical sense?
#    - Is the question clear and unambiguous?

# 2. **Read the Answer:**
#    - Does the answer directly address the question?

# 3. **Evaluate the Question and Answer Pair:**
#    - Does the answer fulfill the informational needs implied by the question?
#    - Is the answer free from irrelevant information?
#    - Does the answer provide a complete response to the question?

# **Example:**

# **Question:** What is the capital of the administrative territorial entity that Kul Mishan is a part of?

# **Expected Answer Characteristics:**
#    - The answer should be the name of a capital city.
#    - It should correctly correspond to the administrative territorial entity that Kul Mishan is a part of.

# **Sample Answer Evaluations:**
#    - **"Ardal."** – Suitable, addresses the question, provides relevant information.
#    - **"Tehran."** – Suitable if Tehran is indeed the capital of the entity that Kul Mishan is a part of.
#    - **"Iran."** – Unsuitable, does not specify the capital.
#    - **"Kul Mishan is in Ardal."** – Suitable, but less direct.

# **Verification Checklist:**

# - [ ] The question is logical and clear.
# - [ ] The answer directly addresses the question.

# **Question:** {question}

# **Answer:** {answer}

# **Response Format:**
# Please provide your evaluation in the following JSON format:
# - "question_valid": true/false,
# - "answer_relevance": true/false,
# - "comments": "Your comments here"
# """
#     return prompt




In [18]:
# Example usage
question = "What is the administrative territorial entity where the home venue of the sports team that Jeff Clifton was a member of is located?"
answer = "Melbourne"
prompt = generate_verification_prompt(question, answer)

In [19]:
print(prompt)


**Prompt Adequacy Verification Test**

**Objective:** Verify that the question is clear and the answer appropriately addresses it.

**Instructions:**

1. **Evaluate the Question:**
   - Is the question clear and logical?
   - Is it unambiguous?

2. **Evaluate the Answer:**
   - Does it directly address the question?
   - Is it free from irrelevant information?
   - Does it provide a complete response?

**Example for Understanding:**

**Example Question:** What is the capital of the administrative territorial entity that Kul Mishan is a part of?

**Expected Answer Characteristics:**
   - The answer should be the name of a capital city.
   - It should correctly correspond to the administrative territorial entity that Kul Mishan is a part of.

**Sample Answer Evaluations:**
   - **"Ardal."** - Suitable, addresses the question, provides relevant information.
   - **"Tehran."** - Suitable if Tehran is indeed the capital of the entity that Kul Mishan is a part of.
   - **"Iran."** - Unsuita

In [20]:
res = query_openai_model(prompt)

In [21]:
res

('{\n  "question_valid": true,\n  "answer_relevance": true,\n  "comments": "The question is clear and logical, asking for the administrative territorial entity associated with the home venue of a sports team that an individual, Jeff Clifton, was a member of. The answer \'Melbourne\' directly addresses this question by naming the location that corresponds to the administrative territorial entity of the sports team\'s home venue. However, without specific knowledge of Jeff Clifton and the sports team he was a member of, it\'s difficult to independently verify the accuracy of \'Melbourne\' as the correct answer, but it is assumed to be correct for this evaluation. The answer is relevant and free from unnecessary information, providing a direct response to the question asked."\n}',
 CompletionUsage(completion_tokens=153, prompt_tokens=373, total_tokens=526))